In [2]:
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy
import pandas as pd
from spacy.matcher import PhraseMatcher

In [4]:
nlp = spacy.load('en_core_web_sm')

In [5]:
df = pd.read_csv("D:\FYP\Excel_files\Text_db_final_3.csv", encoding = 'utf-8')

In [6]:
#df_ageSent = pd.read_csv(r"D:\FYP\Excel_files\ageSent.csv",encoding = 'utf-8')

In [7]:
#ageSent = df_ageSent.to_dict(orient = 'list')

In [8]:
#ageSent

In [6]:
def custom_sentencizer(doc):
    ''' Look for sentence start tokens by scanning for periods only. '''
    for i, token in enumerate(doc[:-2]):  # The last token cannot start a sentence
        if token.text == ".":
            doc[i+1].is_sent_start = True
        else:
            doc[i+1].is_sent_start = False  # Tell the default sentencizer to ignore this token
    return doc

In [7]:
nlp.add_pipe(custom_sentencizer,before = 'parser')

In [8]:
#df.loc[df.name == 'so_23_08_45.']

,Unnamed: 0,name,text
426,426,so_23_08_45.,PURSUEA CAREER OF PRESTIGE & HONOUR JOIN THE S...


#### The patterns used to extract age limit

In [9]:
# to extract all the sentences that has age init
p1 = [{'LEMMA':'age'}]

In [10]:
p2 =  [{'POS':{'IN':['ADP','VERB']}},{'IS_PUNCT': True,'OP':'?'},{'TEXT':{'REGEX':'\d{2}'},'OP':'?'},{'IS_PUNCT': True,'OP':'?'},
           {'TEXT':{'REGEX':'\d{2}'}},{'LOWER':'years','OP':'?'}]

In [11]:
# pattern type - not less than 35 years and not more than 55 years
p3 = [{'LOWER':'not'},{'POS':'ADJ'},{'LOWER':'than'},{'TEXT':{'REGEX':'\d{2}'}},{'LOWER':'years'},
      {'LOWER':'and','OP':'?'},{'LOWER':'not'},{'POS':'ADJ'},{'LOWER':'than'},
     {'TEXT':{'REGEX':'\d{2}'}},{'LOWER':'years'}]

In [12]:
# pattern type - not be more than 64 years ,  Not more than 45 years
p4 = [{'LOWER':'not'},{'POS':'AUX','OP':'?'},{'POS':'ADJ'},{'LOWER':'than'},
     {'TEXT':{'REGEX':'\d{2}'}},{'LOWER':'years'}]

In [13]:
# pattern type - be less than 40 years
p5 = [{'LOWER':'not','OP':'?'},{'POS':'AUX','OP':'?'},{'POS':'ADJ'},{'LOWER':'than'},
     {'TEXT':{'REGEX':'\d{2}'}},{'LOWER':'years'}]

In [14]:
# pattern type - 20 - 34 years , 20 - 34
p6 = [{'TEXT':{'REGEX':'\d{2}'}},{},{'TEXT':{'REGEX':'\d{2}'}},{'LOWER':'years','OP':'?'}]
#p6 = [{'IS_DIGIT':True},{},{'IS_DIGIT':True},{'LOWER':'years','OP':'?'}]


In [15]:
# pattern type - bellow 26 years
p7 = [{'LOWER':'bellow'},{'OP':'?'},{'TEXT':{'REGEX':'\d{2}'}},{'LOWER':'years','OP':'?'}]

In [16]:
#p8 = [{'POS':{'IN':['ADP','ADV']},'OP':'?'},{'IS_DIGIT':True,'OP':'?'},{'IS_DIGIT':True},{'POS':'NOUN'}]
p8 = [{'POS':{'IN':['ADP','ADV']},'OP':'?'},{'IS_DIGIT':True,'OP':'?'},{'IS_DIGIT':True},{'POS':{'IN':['NOUN','PROPN']}}]

In [17]:
# matcher object for pattern 1
matcher = Matcher(nlp.vocab)
matcher.add('Agematch',None,p1)

In [18]:
# Dictionary to put age included sentences
ageSent = {}

In [19]:
# Dictionary to put age not included sentences
ageSentnone = {}

In [20]:
# Function to extract age included sentences
def extractsentAge(index):
    print(index)
    t = df.text[index]
    id_ = df.name[index]
    doc = nlp(t)
    k = matcher(doc)
    try:
        agelist = []
        if len(k) == 1: 
            #print(len(k))
            span = doc[k[0][1]:k[0][2]]
            agelist.append(span.sent)
            ageSent[id_] = agelist
            #print(span.sent)
        elif len(k) > 1:
            #print(len(k))
            #agelist = []
            for i in range(len(k)):
                span = doc[k[i][1]:k[i][2]]
                #print(span.sent)
                agelist.append(span.sent)
            ageSent[id_] = agelist
        else:
            ageSentnone[id_] = None
    except IndexError:
        error = "No age limit"
        ageSentnone[id_] = ""
        

In [21]:
# Looping throgh every row in the df. 
for i in range(len(df.text)):
#for i in range(100):
    try:
        extractsentAge(i)
    except TypeError:
        continue

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


In [32]:
ageSent

{'TJ_C1_ (101).': [Age Below 35 years.],
 'TJ_C1_ (102).': [Contact: > Age below 42 years.],
 'TJ_C1_ (105).': [* Age below 32 years.],
 'TJ_C1_ (107).': [ACCOUNTS EXECUTIVES (MALE) Qualifications and Experience • Part qualification in ICASL/ CIMA/ ACCA • Minimum 3 years working experience as an Accounts Executive • Age below 28 • Ability to use and adapt to ERP & Quick book systems is a distinct advantage.],
 'TJ_C1_ (108).': [* Age bellow 26 years.],
 'TJ_C1_ (109).': [Age between 25-30 Positive attitude and working under pressure Female Candidates only Tel: 0771099056 E-Mail: nuwan85dhammika@gmail.com],
 'TJ_C1_ (110).': [• Should be able to work independently • Male candidates age below 35 • Minimum 2 year experience If you feel you are the right candidate for the above please submit your resume to hr@papyrus.lk within 7 working days.],
 'TJ_C1_ (111).': [V Age below 30 years.],
 'TJ_C1_ (119).': [Requirements • Age below 35 years.],
 'TJ_C1_ (12).': [• Age below 35 Account Executi

In [22]:
# This function takes the below arguments and return two dictionaries. 
# Dict1: patterns extracted , Dict2: senetence which didnt get extracted
# 'n' here is which tuple set I should take from 'k'

def extractagelimit(mat,dictionary,extractedPatterns,notExtractedPatterns,n):
    #counter = 0
    for key,value in dictionary.items(): 
        extractedlimits = []
        notextractedlimits = []
        for i in range(len(value)):  
            try:
                k = mat(value[i])
                extractedlimits.append(value[i][k[n][1]:k[n][2]])
                #ageSentp2p3[key] = value[i][k[-1][1]:k[-1][2]]

            except IndexError:
                notextractedlimits.append(value[i])
                #ageSent_notExtracted[key] = value
                #counter += 1
                #print(count)
        if len(extractedlimits) != 0:
            extractedPatterns[key] = extractedlimits
        if len(notextractedlimits) != 0:
            notExtractedPatterns[key] = notextractedlimits

In [30]:
pattern = [p2,p3]
matcher2 = Matcher(nlp.vocab)
matcher2.add('Agematch',None,*pattern)
# to put sentences extracted using p2 and p3
ageSentp2p3 = {}
# to put non extracted from p2 and p3
ageSent_notExtracted = {}
extractagelimit(matcher2,ageSent,ageSentp2p3,ageSent_notExtracted,-1)

In [31]:
ageSentp2p3

{'TJ_C1_ (101).': [Below 35 years],
 'TJ_C1_ (102).': [below 42 years],
 'TJ_C1_ (105).': [below 32 years],
 'TJ_C1_ (107).': [below 28],
 'TJ_C1_ (109).': [between 25-30],
 'TJ_C1_ (110).': [below 35],
 'TJ_C1_ (111).': [below 30 years],
 'TJ_C1_ (119).': [below 35 years],
 'TJ_C1_ (12).': [below 35],
 'TJ_C1_ (120).': [of 25 - 40 years],
 'TJ_C1_ (122).': [below 30 years],
 'TJ_C1_ (131).': [between 21– 32 years, between 18 -30 years],
 'TJ_C1_ (135).': [Below 35 years],
 'TJ_C1_ (140).': [BELOW 40],
 'TJ_C1_ (143).': [below 35 years],
 'TJ_C1_ (151).': [of 23 – 35 years],
 'TJ_C1_ (158).': [below 23 years],
 'TJ_C1_ (16).': [before 18h, before 18h],
 'TJ_C1_ (163).': [below 25 years],
 'TJ_C1_ (167).': [below 35 years],
 'TJ_C1_ (17).': [below 30 Years],
 'TJ_C1_ (170).': [below 30],
 'TJ_C1_ (171).': [below 30 Years],
 'TJ_C1_ (173).': [between 23-28],
 'TJ_C1_ (176).': [below 30 Years, below 25 Years, below 30 Years],
 'TJ_C1_ (180).': [between 20 - 30 years],
 'TJ_C1_ (181).': [B

In [29]:
ageSentp2p3

{'TJ_C1_ (101).': [Below 35],
 'TJ_C1_ (102).': [below 42],
 'TJ_C1_ (105).': [below 32],
 'TJ_C1_ (107).': [below 28],
 'TJ_C1_ (109).': [between 25],
 'TJ_C1_ (110).': [below 35],
 'TJ_C1_ (111).': [below 30],
 'TJ_C1_ (119).': [below 35],
 'TJ_C1_ (12).': [below 35],
 'TJ_C1_ (120).': [of 25],
 'TJ_C1_ (122).': [below 30],
 'TJ_C1_ (131).': [between 21, between 18],
 'TJ_C1_ (135).': [Below 35],
 'TJ_C1_ (140).': [BELOW 40],
 'TJ_C1_ (143).': [below 35],
 'TJ_C1_ (151).': [of 23],
 'TJ_C1_ (158).': [below 23],
 'TJ_C1_ (16).': [above 35, above 35],
 'TJ_C1_ (163).': [below 25],
 'TJ_C1_ (167).': [below 35],
 'TJ_C1_ (17).': [below 30],
 'TJ_C1_ (170).': [below 30],
 'TJ_C1_ (171).': [below 30],
 'TJ_C1_ (173).': [between 23],
 'TJ_C1_ (176).': [below 30, below 25, to 25,000/=],
 'TJ_C1_ (180).': [between 20],
 'TJ_C1_ (181).': [Below 30],
 'TJ_C1_ (182).': [below 28],
 'TJ_C1_ (186).': [between 25],
 'TJ_C1_ (19).': [below 30],
 'TJ_C1_ (194).': [of 20],
 'TJ_C1_ (195).': [below 40]

In [35]:
matcher3 = Matcher(nlp.vocab)
matcher3.add('Agematch',None,p4)
# to put sentences extracted using p4
ageSentp4 = {}
# to put not extracted sentences from p4
ageSent_notExtracted_2 = {}
# extracting pattern 4
extractagelimit(matcher3,ageSent_notExtracted,ageSentp4,ageSent_notExtracted_2,-1)

In [36]:
matcher4 = Matcher(nlp.vocab)
matcher4.add('Agematch',None,p5)
# to put sentences extracted using p4
ageSentp5 = {}
# to put not extracted sentences from p2 and p3
ageSent_notExtracted_3 = {}
# extracting pattern 5
extractagelimit(matcher4,ageSent_notExtracted_2,ageSentp5,ageSent_notExtracted_3,-1)

In [37]:
matcher5 = Matcher(nlp.vocab)
matcher5.add('Agematch',None,p6)
# to put sentences extracted using p6
ageSentp6 = {}
# to put not extracted sentences from p6
ageSent_notExtracted_4 = {}
# extracting pattern 6
extractagelimit(matcher5,ageSent_notExtracted_3,ageSentp6,ageSent_notExtracted_4,-1)

In [38]:
matcher6 = Matcher(nlp.vocab)
matcher6.add('Agematch',None,p7)
# to put sentences extracted using p7
ageSentp7 = {}
# to put not extracted sentences from p7
ageSent_notExtracted_5 = {}
# extracting pattern 7
extractagelimit(matcher6,ageSent_notExtracted_4,ageSentp7,ageSent_notExtracted_5,-1)

In [39]:
ageSentp7

{'TJ_C1_ (108).': [bellow 26 years],
 'TJ_C1_ (139).': [bellow 30 years],
 'TJ_V1_47.': [bellow 35 years],
 'SO_02_02_11.': [bellow 45 years]}

In [40]:
def extractagelimitPhrase(mat,dictionary,extractedPatterns,notExtractedPatterns):
    #counter = 0
    for key,value in dictionary.items(): 
        extractedlimits = []
        notextractedlimits = []
        for i in range(len(value)):  
            try:
                k = mat(nlp(str(value[i])))
                extractedlimits.append(value[i][k[-1][1]:k[-1][2]])
                #ageSentp2p3[key] = value[i][k[-1][1]:k[-1][2]]

            except IndexError:
                notextractedlimits.append(value[i])
                #ageSent_notExtracted[key] = value
                #counter += 1
                #print(count)
        if len(extractedlimits) != 0:
            extractedPatterns[key] = extractedlimits
        if len(notextractedlimits) != 0:
            notExtractedPatterns[key] = notextractedlimits

In [41]:
matcher7 = PhraseMatcher(nlp.vocab)
# terms that should be matched
terms = ['Maximum age limit will not be applicable to internal applicants',
         'The upper age limit will not apply to the internal candidates',
        'upper age limit will not apply to the internal',
        'The upper age limit will not be applicable for those who are employed in the government sector',
        'without regard',
        'wothout discrimination',
        'Maximum age limit is not applicable for internal applicants',
        'without discrimination',
        'not discriminate',
        'all ages',
        'will not apply to the candidates in the Government Sector Organizations.',
        'Internal Applicants are not affected by the upper age limit.',
        'The upper age limit will not be applicable to Internal Candidates.',
        'no barrier']

patterns = [nlp.make_doc(text) for text in terms]
matcher7.add("Nolimit",None, *patterns)

In [42]:
# to put sentences extracted using phrase matcher
ageSentPhrase = {}
# to put not extracted sentences from phrase matcher
ageSent_notExtracted_6 = {}
# extracting pattern 7
extractagelimitPhrase(matcher7,ageSent_notExtracted_5,ageSentPhrase,ageSent_notExtracted_6)

In [43]:
#ageSentPhrase

In [44]:
ageSent_notExtracted_6

{'dn_30_04_1.': [Age : A self-prepared application should be sent by registered post addressed to the Chairman, Sri Lanka Ayurvedic Drugs Corporation, No.],
 'so_02_08_3.': [The ideal candidate should: • Be below the age 45 years with pleasing Personality.],
 'so_09_08_17.': [(Maximum age limit doesn't apply for limited requirements) 02.],
 'so_09_08_25.': [Age limit will not be applicable for applicants from Higher Education Institutes, Government Departments, Corporations and Statutory Boards and they should channel their applications through the respective Heads of institutions.],
 'so_16_08_2.': [17/2016 Age: (II) Having obtained a certificate of proficiency not below than the N.V.Q Level5 issued by a Technical/ Vocational Training Institute accepted by the Tertiary and Vocational Education Commission or recognized Institution in the field of Information Communication and Multimedia Technology/ Information and Communication Technology/ Information Technology or Equivalent.,
  Age: 

In [45]:
matcher8 = Matcher(nlp.vocab)
matcher8.add('Agematch',None,p8)
# to put sentences extracted using p8
ageSentp8 = {}
# to put not extracted sentences from p8
ageSent_notExtracted_7 = {}
# extracting pattern 8
extractagelimit(matcher8,ageSent_notExtracted_6,ageSentp8,ageSent_notExtracted_7,0)

In [46]:
ageSent_notExtracted_7

{'dn_30_04_1.': [Age : A self-prepared application should be sent by registered post addressed to the Chairman, Sri Lanka Ayurvedic Drugs Corporation, No.],
 'so_09_08_17.': [(Maximum age limit doesn't apply for limited requirements) 02.],
 'so_09_08_25.': [Age limit will not be applicable for applicants from Higher Education Institutes, Government Departments, Corporations and Statutory Boards and they should channel their applications through the respective Heads of institutions.],
 'so_16_08_2.': [17/2016 Age: (II) Having obtained a certificate of proficiency not below than the N.V.Q Level5 issued by a Technical/ Vocational Training Institute accepted by the Tertiary and Vocational Education Commission or recognized Institution in the field of Information Communication and Multimedia Technology/ Information and Communication Technology/ Information Technology or Equivalent.,
  Age: Qualifications: (i) Ability to read and write.],
 'so_16_08_36.': [The upper age limit will not be app

In [47]:
ageSentp8

{'so_02_08_3.': [45 years],
 'so_30_08_46.': [22 -26],
 'TJ_M1_56.': [3 years],
 'SO_02_02_13.': [40 years],
 'SO_09_02_44.': [20 25 Years],
 'SO_16_02_63.': [around 35 years],
 'TJ_L1_40.': [around 25 years],
 'TJ_x1_28.': [30 years],
 'TJ_A1_3.': [4 years],
 'TJ_A1_489.': [28 years],
 'TJ_A1_88.': [25 -30],
 'SO_19_01_116.': [5 years],
 'SO_19_01_139.': [15 years],
 'so_05_07_16.': [18 years],
 'so_05_07_40.': [65 years],
 'so_12_07_10.': [around 45 years],
 'so_12_07_18.': [below 30 Yrs],
 'so_19_07_2.': [35%],
 'so_28_06_11.': [preferably 55 years],
 'TJ_H1_54.': [around 25 years],
 'TJ_E1_257.': [BELOW 35 YEARS],
 'so_20_09_15.': [1 Rs],
 'so_20_09_4.': [58 years],
 'so_27_09_41.': [35 Years]}

In [208]:
# function to convert above created dictionary into df and get it saved.
# Please note: Have to specify the path and the file name that should be saved as
def dicttodf(dictnm):  
    for k,value in dictnm.items():
        dictnm[k] = str(value)
    df = pd.DataFrame.from_dict(dictnm,orient = 'index')
    df.to_csv(r'D:\FYP\Excel_files\ageSentnotextractedafterp6.csv', encoding = 'utf-8')

In [209]:
dicttodf(ageSent_notExtracted_4)

In [27]:
df

,Unnamed: 0,name,text
0,0,TJ_C1_ (1).,mûve ASSOCIATE FINANCE WHO WE ARE Muve is a UK...
1,1,TJ_C1_ (10).,ACCOUNTANT / ACCOUNTS EXECUTIVE Orion Holdings...
2,2,TJ_C1_ (100).,ASSISTANT ACCOUNTANT Westgate International (P...
3,3,TJ_C1_ (101).,ACCOUNTS ASSISTANT A Well-established BOI and ...
4,4,TJ_C1_ (102).,A Well-established ISO ASSISTANT ACCOUNTANT Ce...
...,...,...,...
3530,3530,TJ_U1_ (5).,VACANCIES IN OUR SF HORANΑ ΒΟΙ PROJECT Industr...
3531,3531,TJ_U1_ (6).,SRI LANKA'S NUMBER ONE BRAND IS COUNTING ON YO...
3532,3532,TJ_U1_ (7).,"DPJH Drive, Determination, Diversity Assistant..."
3533,3533,TJ_U1_ (8).,Career Opportunity. Quality Assurance Executiv...


In [68]:
# converting the dict values to strings. otherwise it wont get converted to a df
#for k,value in ageSent_notExtracted_1.items():
#    ageSent_notExtracted_1[k] = str(value)

In [69]:
#df = pd.DataFrame.from_dict(ageSent_notExtracted_1,orient = 'index')

In [70]:
#df.to_csv(r"D:\FYP\Excel_files\agelimit_analysis_after_p2p3p4.csv", encoding = 'utf-8')